#  DenseNet - 201 Model

## Part-1 Data Preprocessing

In [2]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [3]:
tf.__version__

'2.3.1'

In [4]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
training_set = train_datagen.flow_from_directory(
        'dataset/training_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')
validation_set = train_datagen.flow_from_directory(
        'dataset/validation_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

Found 2723 images belonging to 2 classes.
Found 290 images belonging to 2 classes.


In [5]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(
        'dataset/test_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

Found 1060 images belonging to 2 classes.


In [6]:
x,y = test_set.next()
x.shape

(32, 64, 64, 3)

## Part-2 Building the Model

In [27]:
model = tf.keras.models.Sequential()

In [28]:
model.add(tf.keras.applications.DenseNet201(include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,))
model.output

<tf.Tensor 'densenet201/relu/Relu_3:0' shape=(None, None, None, 1920) dtype=float32>

In [29]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet201 (Functional)     (None, None, None, 1920)  18321984  
Total params: 18,321,984
Trainable params: 18,092,928
Non-trainable params: 229,056
_________________________________________________________________


## Part-3 Training the Model

In [30]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [31]:
model.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25


InvalidArgumentError:  Incompatible shapes: [32,1] vs. [32,2,2,1920]
	 [[node gradient_tape/binary_crossentropy/mul/BroadcastGradientArgs (defined at <ipython-input-31-95720dd6c412>:1) ]] [Op:__inference_train_function_215018]

Function call stack:
train_function


In [ ]:
import matplotlib.pyplot as plt
(history.history.keys())

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
test_loss, test_acc = model.evaluate(test_set, verbose=2)
print('\nTest Accuracy: ', test_acc)

In [ ]:
import pandas as pd
import seaborn as sn
import tensorflow as tf

model = tf.keras.models.load_model("planet.h5")
filenames = test_set.filenames
nb_samples = len(test_set)
y_prob=[]
y_act=[]
test_set.reset()
for _ in range(nb_samples):
    X_test,Y_test = test_set.next()
    y_prob.append(model.predict(X_test))
    y_act.append(Y_test)
    
predicted_class = [list(training_set.class_indices.keys())[i.argmax()] for i in y_prob]
actual_class = [list(training_set.class_indices.keys())[i.argmax()] for i in y_act]

out_df =pd.DataFrame(np.vstack([predicted_class,actual_class]).T,columns=['predicted_class','actual_class'])
confusion_matrix = pd.crosstab(out_df['actual_class'],out_df['predicted_class'],rownames=['Actual'], colnames=['Predicted'])

sn.heatmap(confusion_matrix, cmap='Blues', annot=True,fmt='d')
plt.show()
print('test_accuracy: {}'.format((np.diagonal(confusion_matrix).sum()/confusion_matrix.sum().sum()*100)))

## Part-4 Single Prediction

In [14]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/check_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'unhealthy'
else:
  prediction = 'healthy'

In [15]:
print(prediction)

healthy
